In [1]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [2]:
# Define the model
EMB_SIZE = 64
HID_SIZE = 64
HID_LAY = 2
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_h = [model.add_parameters((HID_SIZE, EMB_SIZE if lay == 0 else HID_SIZE)) for lay in range(HID_LAY)]
b_h = [model.add_parameters((HID_SIZE)) for lay in range(HID_LAY)]
W_sm = model.add_parameters((ntags, HID_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                    # Softmax bias

In [3]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  h = dy.esum([dy.lookup(W_emb, x) for x in words])
  for W_h_i, b_h_i in zip(W_h, b_h):
    h = dy.tanh( dy.parameter(W_h_i) * h + dy.parameter(b_h_i) )
  return dy.parameter(W_sm) * h + dy.parameter(b_sm)

In [4]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5533, time=0.95s
iter 0: test acc=0.3801
iter 1: train loss/sent=1.2708, time=0.95s
iter 1: test acc=0.4158
iter 2: train loss/sent=1.0447, time=0.97s
iter 2: test acc=0.3919
iter 3: train loss/sent=0.8109, time=0.95s
iter 3: test acc=0.4023
iter 4: train loss/sent=0.6108, time=0.97s
iter 4: test acc=0.3860
iter 5: train loss/sent=0.4319, time=0.98s
iter 5: test acc=0.3923
iter 6: train loss/sent=0.3139, time=0.94s
iter 6: test acc=0.4005
iter 7: train loss/sent=0.2178, time=0.99s
iter 7: test acc=0.3869
iter 8: train loss/sent=0.1589, time=0.97s
iter 8: test acc=0.3905
iter 9: train loss/sent=0.1129, time=0.98s
iter 9: test acc=0.3842
iter 10: train loss/sent=0.0868, time=0.99s
iter 10: test acc=0.3819
iter 11: train loss/sent=0.0684, time=0.96s
iter 11: test acc=0.3910
iter 12: train loss/sent=0.0536, time=0.97s
iter 12: test acc=0.3824
iter 13: train loss/sent=0.0442, time=0.98s
iter 13: test acc=0.3910
iter 14: train loss/sent=0.0364, time=0.96s
iter 14: t

KeyboardInterrupt: 